<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/notebooks/2021Roelofs_Conceptual_bias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- title: A unified computational account of cumulative semantic, semantic blocking, and semantic distractor effects in picture naming
- Journal: Cognition
- author: Ardi Roelofs
- date: November 2017
- doi: <https://doi.org/10.1016/j.cognition.2017.12.007>
- Original filename: CONCEPTUAL BIAS.C
- url: https://osf.io/6ysp2/


# 概念バイアスシミュレーション, Roelofs (2017)

- original source: Conceptual_Bias.c
- filename: 2021Roelofs_Conceptual_bias.ipynb
- date: 2020-0101
- author: Shin Asakawa <asakawa@ieee.org>

## 概要

絵画命名課題における語彙選択の反応時間のシミュレーションモデルを 3 つの課題で検証: 
1. 連続命名課題
2. 条件群ごと命名課題
3. 絵画-言語 意味干渉課題
上記 3 課題の統一的な計算論的説明を WEAVER++ モデル(Levelt, Roelofs, & Meyer, 1999) を用いて試みた。
概念バイアスを含めることで、意味干渉効果，累積的な意味効果，意味的ブロッキング効果を説明。

前提：
* 競争による語彙選択
* 意味的効果の概念的起源と語彙的機嫌
心理実験データと神経画像学的証拠を取り上げ，3 課題の意味効果の統一的な説明

- note: SIMULATION OF LEMMA RETRIEVAL IN SPEAKING,  Simulation of cumulative semantic and semantic blocking effects
- 要約:
    > 音声言語生成における語彙選択の計算モデルは，連続呼称，ブロック循環呼称，画像-単語干渉の各パラダイムで得られた画像-単語呼称応答時間における意味干渉効果に適用されてきた。
しかし，この 3 つのパラダイムの効果に関する統一的な計算論的説明は欠如している。
ここでは，WEAVER++ モデル (Levelt, Roelofs, & Meyer, 1999) に概念的バイアスを含めることで，意味的混乱効果に関するモデルの説明を維持したまま，意味累積効果および意味ブロック効果を説明できることを示す。
このモデルの主要な仮定は以下の通りである。
(1) 競合による語彙選択 (2) 意味効果の概念的起源と語彙的位置。
この説明の概念実証として，コンピュータ・シミュレーションの結果を報告し，行動学的，神経画像学的な証拠に言及する。
この仮定は，この分野の悲観的な見解に反して，3 つのパラダイムにおける意味効果の統一的な説明として十分である。

<!-- Computational models of lexical selection in spoken word production have been applied to semantic interference effects in picture naming response times obtained with continuous naming, blocked-cyclic naming, and picture-word interference paradigms. 
However, a unified computational account of the effects in the three paradigms is lacking. 
Here, I show that the inclusion of conceptual bias in the WEAVER++ model (Levelt, Roelofs, & Meyer, 1999) explains cumulative semantic and semantic blocking effects while preserving the model's account of semantic distractor effects. 
The key assumptions of the account are 
(1) lexical selection by competition, and (2) a conceptual origin and lexical locus of the semantic effects. 
I provide a proof of concept of the account by reporting computer simulation results, addressing behavioral and neuroimaging evidence. 
The assumptions are sufficient for a unified account of semantic effects in the three paradigms, contrary to pessimistic views of this area.-->


In [ ]:
import numpy as np
import sys
import os

# 表示精度桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:7.4f}'.format})
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})
np.set_printoptions(suppress=False, formatter={'float': '{:4.2f}'.format})

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG, display
#display(SVG(filename='../figures/2018Roelofs_fig1.svg'))
#display(SVG(filename='../figures/2018Roelofs_fig3.svg'))

print('概念モデル')
display(SVG(url='https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/master/figures/2018Roelofs_fig3.svg'))

print('\n\n\n実験手続きの外観')
display(SVG(url='https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/master/figures/2018Roelofs_fig1.svg'))


#display(SVG(url='https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/master/figures/2018Roelofs_fig4ab.svg'))
#display(SVG(url='https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/master/figures/2018Roelofs_fig4cd.svg'))
#display(SVG(url='https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/master/figures/2018Roelofs_fig5.svg'))


print('以下に元となった Blake (2013) の実験結果を示す')
display(SVG(url='https://raw.githubusercontent.com/project-ccap/project-ccap.github.io/master/figures/2013Belke_fig2.svg'))



In [ ]:
STEP_SIZE = 25
N_STEPs = int((1000+STEP_SIZE)/STEP_SIZE)  # /* trunc after 41 steps */
N_CONCEPTs = 6  # fork, spoon, cup, glass, knife, manmade 
N_LEMMAs = 6    # Gabel, Loffel, Tasse, Glas, Messer, KUENSTLICH
N_GENDERs = 3   # 男性，女性，中性
N_BUTTONs = 2   # 人工物，自然物
N_TASKs = 5     # 絵画命名，絵画分類，単語命名，冠詞付き単語命名，絵画命名fMRI
N_POSs = 5      # ordinal position
Y = 1.0         # connection present
N = 0.0         # connection absent
PICTURE_NAMING = 0          # Belke (2013, JML), Exp 1.
PICTURE_CLASSIFICATION = 1  # manmade, natural
WORD_NAMING = 2
WORD_NAMING_WITH_DET = 3
PICTURE_NAMING_fMRI = 4 # BOLD response, Canini et al. (2016, HBM)
task_l = ('PICTURE_NAMING',  'PICTURE_CLASSIFICATION',  'WORD_NAMING',\
          'WORD_NAMING_WITH_DET', 'PICTURE_NAMING_fMRI')

#PICTURE_NAMING, PICTURE_CLASSIFICATION, WORD_NAMING, \
#WORD_NAMIMG_WITH_DET, PICTURE_NAMING_fMRI = 0, 1, 2, 3, 4
tasks = ('PICTURE_NAMING', 'PICTURE_CLASSIFICATION', 'WORD_NAMING', 
         'WORD_NAMIMG_WITH_DET', 'PICTURE_NAMING_fMRI')


# Labeling network nodes
# Concepts
FORK, SPOON, CUP, GLASS, KNIFE, MANMADE = 0, 1, 2, 3, 4, 5
concept_l = ('FORK', 'SPOON', 'CUP', 'GLASS', 'KNIFE', 'MANMADE')

# German lemmas
#fork, sppon,    cup, glass, knife, manmake
GABEL, LOFFEL, TASSE, GLAS, MESSER, KUENSTLICH = 0, 1, 2, 3, 4, 5
lemma_l = ('GABEL', 'LOFFEL', 'TASSE', 'GLAS', 'MESSER', 'KUENSTLICH')

# Grammatical genders
# masculine, feminine, neuter
MASC, FEM, NEUTER = 0, 1, 2
gender_l = ('MASC', 'FEM', 'NEUTER')

# /* Button nodes */
B_MANMADE, B_NATURAL = 0, 1  # Button 1: manmade, Button 2: natural
button_l = ('B_MANMADE', 'B_NATURAL')

In [ ]:
def make_weights(N_CONCEPTs=N_CONCEPTs):
    CONC_con = np.zeros((N_CONCEPTs, N_CONCEPTs), dtype=np.float)  # double CONC_con[N_CONCEPTs][N_CONCEPTs];

    # Connections conceptual network for categorical relations
    CATEG_con = np.array([
        [N, Y, Y, Y, Y, Y],  # FORK
        [Y, N, Y, Y, Y, Y],  # SPOON
        [Y, Y, N, Y, Y, Y],  # CUP
        [Y, Y, Y, N, Y, Y],  # GLASS
        [Y, Y, Y, Y, N, Y],  # KNIFE
        [Y, Y, Y, Y, Y, N]   # MANMADE
    ])

    # Connections conceptual network for thematic relations
    ASSOC_con = np.array([
        [N, N, N, N, N, Y],  # CHURCH
        [N, N, N, N, N, Y],  # RING 
        [N, N, N, N, N, Y],  # DRESS
        [N, N, N, N, N, Y],  # CAKE
        [N, N, N, N, N, Y],  # BOUQUET
        [Y, Y, Y, Y, Y, N],  # WEDDING
    ])

    # Connections between concept and lemma nodes
    # Lemma nodes are Gabel, Loffel, Tasse, Glas, Messer (and kuenstlich [manmade]) 
    # for the category "tableware" of Belke (2013, JML), 
    # and Kirche, Ring, Brautkleid, Torte, Blumenstrauss (and Hochzeit) for the theme "wedding"
    # of Rose & Abdel Rahman (2016, Cognition)
    LEM_con = np.array([
        [Y, N, N, N, N, N],  # Gabel フォーク  ，教会
        [N, Y, N, N, N, N],  # Loffel スプーン ，指輪
        [N, N, Y, N, N, N],  # Tasse カップ   ，ドレス
        [N, N, N, Y, N, N],  # Glas グラス    ，ケーキ
        [N, N, N, N, Y, N],  # Messer ナイフ  ，花束
        [N, N, N, N, N, Y],  # Kuenstlich 人工物，結婚式
    ])

    # Connections between lemma and gender nodes
    GEN_con = np.array([ # MASC, FEM, NEUTER
        [N, Y, N],  # GABEL フォークは女性
        [Y, N, N],  # LOFFEL スプーンは男性
        [N, Y, N],  # TASSE  カップは女性
        [N, N, Y],  # GLAS   グラスは中性
        [N, N, Y],  # MESSER ナイフは中性
        [N, N, N],  # KUENSTLICH
    ])

    # Connections between concept and button nodes
    BUTTON_con = np.array([ # B_MANMADE B_NATURAL
        [N, N], # Fork
        [N, N], # Spoon
        [N, N], # Cup
        [N, N], # Glass
        [N, N], # Knife
        [Y, N], # Manmade
    ])
    
    weights = {'CONC': CONC_con,
               'CATEG': CATEG_con,
               'ASSOC': ASSOC_con,
               'LEM': LEM_con,
               'GEN': GEN_con,
               'BUTTON': BUTTON_con}
    
    return weights


def make_layers():
    return {'concept': node(N_CONCEPTs), 
            'lemma': node(N_LEMMAs), 
            'gender': node(N_GENDERs), 
            'button': node(N_BUTTONs)}
    

In [ ]:
pos = 0  # int pos;   /* ordinal position, i.e., 1 to 5 (in C is 0 to 4) */
T = 0    # int T;     /* time in ms */
#task = 0 # int task;  /* PICTURE NAMING, PICTURE CLASSIFICATION, WORD NAMING, 
#         # WORD NAMIMG WITH DET, PICTURE_NAMING_fMRI */

# Parameter set plus values
CONC_rate = 0.5 * 0.0101 * STEP_SIZE  # double CONC_rate = 0.5 * 0.0101 * STEP_SIZE;   
                                      # 0.5 of default prop per step_size ms 
                                      # because of the increased size of conceptual
                                      # network, cf. Roelofs (2003, Psych. Rev.) */
LEM_rate = 0.0074 * STEP_SIZE   # prop per step_size ms
DECAY_rate = 0.0240 * STEP_SIZE # prop per step_size ms
EXTIN = 0.1965 * STEP_SIZE      # act_units per step_size ms
CRIT_DIFF = 0.0                 # act_units
BIAS = 2.5                      #
BLOCKED_CYCLIC = False          # 1 = run for blocked-cyclic naming
PREV_BIAS = 0.0                 # previous bias
# The PREV_BIAS parameter is needed for simulating blocked-cyclic
# naming, because the bias increase during a cycle is also present 
# during the next cycle. To run the model for blocked-cyclic naming,
# set the PREV_BIAS values by hand for each new cycle.
# Previous bias in 
# cycle 1: 0.0, 
# cycle 2: 2.5, 
# cycle 3: 5.0, 
# cycle 4: 7.5, 
# cycle 5: 10.0, 
# cycle 6: 12.5
# The semantic blocking effect in a cycle is the mean increase over
# ordinal positions relative to the first position. The effect of bias
# for the first ordinal position is the same for the homogeneous and 
# the heterogeneous conditions. For later ordinal positions, the bias 
# effect will differ between conditions because the position effect
# occurs only within categories or themes (thus only in the homogeneous
# condition).

class node():
    """# Here a network node is defined, having an activation level,
    # and an input buffer; its output is equal to its activation
    # struct node { 
    #	 double act;    # 出力
    #	 double input;  # 入力
    # } concept[N_CONCEPTs], lemma[N_LEMMAs], gender[N_GENDERs], button[N_BUTTONs];
    """
    def __init__(self, n):
        self.input = np.zeros((n,), dtype=np.float)
        self.output = np.zeros((n,), dtype=np.float)

    def update(self, decay=DECAY_rate):
        self.output = (1.0 - decay) * self.output + self.input
    
    def __len__(self):
        return len(self.input)
    
    def clear_all(self):
        self.input.fill(0.)
        self.output.fill(0.)
        
    def clear_output(self):
        self.output.fill(0.)
        
    def clear_input(self):
        self.input.fill(0.)
        

#TAXONOMIC = 1  # categorical relatedness, Belke (2013, Journal of Memory and Language)
#THEMATIC = 0   # thematic relatedness, Rose & Abdel Rahman (2016, Cognition)

# The parameter values for the critical difference [selection threshold] 
# for the different tasks: default: all values the same
CRITICAL_DIFFS = {
    'CD_PN': 1.6,  # Picture naming
    'CD_PC': 1.6,  # Picture classification
    'CD_WR': 1.6,  # Word reading  
}
#CD_PN = 1.6  # Picture nameing
#CD_PC = 1.6  # Picture classification
#CD_WR = 1.6  # Word reading 

In [ ]:
#/*****************
# * MAIN ROUTINES *
# *****************/
def main(relation='TAXONOMIC', Critical_Diffs=CRITICAL_DIFFS, graph=False):  
    # relation = {'TAXONOMIX' or 'THEMATIC'}
    # TAXONOMIC: categorical relatedness, Belke (2013, Journal of Memory and Language)
    # THEMATIC: thematic relatedness, Rose & Abdel Rahman (2016, Cognition)
    
    # The parameter values for the critical difference [selection threshold] 
    # for the different tasks: default: all values the same
    # CRITICAl_DIFFS = {
    #  'CD_PN': 1.6,  # Picture naming
    #  'CD_PC': 1.6,  # Picture classification
    #  'CD_WR': 1.6,  # Word reading  
    # }

    SIM = np.zeros((N_TASKs, N_POSs), dtype=np.float)
    BOLD = np.zeros((N_TASKs, N_POSs), dtype=np.float)
    
    # control flags for production rule application, 
    # cf. EPIC, Meyer & Kieras, Psych. Rev. 1997
    flags = {'CONCEPT': False, 
             'ENHANCE_CONCEPT': False, 
             'LEMMA': False, 
             'GENDER': False, 
             'CATEGORY_CONCEPT': False, 
             'BUTTON_NODE': False }
    
    W = make_weights()
    Layers = make_layers()
    
    for task in tasks:
        i_task = tasks.index(task)
        # 課題 task を順に実行
        #print_heading()
        
        # 課題ごとのパラメータ設定
        # CRIT_DIFF: Critical difference の意味
        CRIT_DIFF = set_task_dependent_parameters(task, Critical_Diffs)
        #SIM, BOLD = reset_system(task, SIM, BOLD)
        SIM[i_task].fill(0); BOLD[i_task].fill(0.)
        
        # 結合係数行列の設定
        W = set_spreading_rates(W, relation='TAXONOMIC')
        
        # シミュレーション本体
        SIM, BOLD = compute_POS_functions(task, Layers, W, flags, SIM, BOLD, CRIT_DIFF)
        print_expectations_of_RT(task, SIM, BOLD, graph=graph)
        
        # 結合係数行列をリセット
        W = reset_spreading_rates(W, CONC_rate, LEM_rate)


def compute_POS_functions(task, Layers, W, flags, SIM, BOLD, CRIT_DIFF):
    """シミュレーション本体"""
    mass = np.zeros((N_STEPs,), dtype=np.float)
    hazard = np.zeros((N_STEPs,), dtype=np.float)
    Survival = np.zeros((N_STEPs,), dtype=np.float)
    
    i_task = tasks.index(task)

    for pos in range(N_POSs):  # 各位置ごとに繰り返す
        mass, hazard, Survival = reset_f_h_S_(mass, hazard, Survival)
        Layers = reset_network(Layers)
        flags = reset_production_rule_system(flags)
        
        # stop for lemma retrieval latency > 1000 ms
        for T in range(0, 1000, STEP_SIZE):
            hazard = compute_hazard_rate(T, task, flags, Layers, W, hazard, CRIT_DIFF)
            Layers = update_network(task, pos, Layers, W, flags)
            flags = apply_production_rules(task, flags, Layers)
        
        Survival = compute_cumul_survival_function(hazard, Survival)
        mass = compute_mass_function(mass, hazard, Survival)
        SIM[i_task][pos] = compute_expectation_of_RT_(mass)

    if task == 'PICTURE_NAMING_fMRI':
        BOLD = compute_BOLD(task, SIM, BOLD)
    return SIM, BOLD

    
def reset_system(task, SIM, BOLD):
    i = task_l.index(task)
    for j in range(N_POSs):
        SIM[i][j] = 0

    for j in range(N_POSs):
        BOLD[i][j] = 0.0
        
    return SIM, BOLD


def set_spreading_rates(W, relation='TAXONOMIC'):

    # Sets the conceptual connections for categorical reations
    if relation == 'TAXONOMIC':
        W['CONC'] = np.copy(W['CATEG'])

    # Sets the conceptual connections for thematic reations
    if relation == 'THEMATIC':
        W['CONC'] = np.copy(W['ASSOC'])

    # Sets the strengths of the connections
    W['CONC'] *= CONC_rate
    W['LEM'] *= LEM_rate
    W['GEN'] *= LEM_rate
    W['BUTTON'] *= LEM_rate
    
    return W


def set_task_dependent_parameters(task, Critical_Diffs):

    if task == 'PICTURE_NAMING':
        return Critical_Diffs['CD_PN']
    elif task == 'PICTURE_CLASSIFICATION':
        return Critical_Diffs['CD_PC']
    elif task == 'WORD_NAMING':
        return Critical_Diffs['CD_WR']
    elif task == 'WORD_NAMING_WITH_DET':
        return Critical_Diffs['CD_WR']
    elif task == 'PICTURE_NAMING_fMRI':
        return Critical_Diffs['CD_PN']


def reset_spreading_rates(W, CONC_rate, LEM_rate):
    """
    Needed, because otherwise in later tasks rates become a 
    proportion of earlier ones; this routine keeps them fixed
    """
    W['CONC'] *= 1.0/CONC_rate
    W['LEM'] *= 1.0/LEM_rate
    W['GEN'] *= 1.0/LEM_rate
    W['BUTTON'] *= 1.0/LEM_rate
    
    return W
        

def reset_network(Layers):
    for x in Layers:
        Layers[x].clear_output()
    return Layers


def reset_production_rule_system(flags):
    """reset control flags in working memory"""
    flags['CONCEPT'] = False
    flags['ENHANCE_CONCEPT'] = False
    flags['LEMMA'] = False
    flags['GENDER'] = False
    flags['CATEGORY_CONCEPT'] = False
    flags['BUTTON_NODE'] = False
    return flags


def apply_production_rules(task, flags, Layers):

    # Production rule application for picture naming
    # here the lemma of Gabel for the selected concept FORK is flagged
    if (task=='PICTURE_NAMING' or task=='PICTURE_NAMING_fMRI') \
    and flags['CONCEPT'] and (Layers['lemma'].output[GABEL] > 0):
        flags['LEMMA'] = True

    # here the concept FORK is flagged for selection and enhancement 
    #          cf. P1 in Roelofs, 2003, Psych. Rev., p. 100
    if (task=='PICTURE_NAMING' or task=='PICTURE_NAMING_fMRI') \
    and Layers['concept'].output[FORK]:
        flags['CONCEPT'] = True
        flags['ENHANCE_CONCEPT'] = True

    # production rule application for picture classification 
    # here the button node for the category concept MAN-MADE is flagged 
    if task=='PICTURE_CLASSIFICATION' and flags['CATEGORY_CONCEPT'] \
    and (Layers['button'].output[B_MANMADE] > 0):
        flags['BUTTON_NODE'] = True

    # here the category concept MAN-MADE for the concept FORK is flagged 
    if task=='PICTURE_CLASSIFICATION' \
    and flags['CONCEPT'] \
    and (Layers['concept'].output[MANMADE] > 0):
        flags['CATEGORY_CONCEPT'] = True

    # here the concept FORK is flagged
    if task=='PICTURE_CLASSIFICATION' \
    and (Layers['concept'].output[FORK] > 0):
        flags['CONCEPT'] = True

    # Production rule application for word reading 
    # here the lemma of Gabel is flagged 
    if task=='WORD_NAMING' and (Layers['lemma'].output[GABEL] > 0):
        flags['LEMMA'] = True

    # Production rule application for word reading with gender-marked determiner 
    # here the gender of the lemma of Gabel is flagged 
    if task=='WORD_NAMING_WITH_DET' \
    and flags['LEMMA'] \
    and (Layers['gender'].output[FEM] > 0):
        flags['GENDER'] = True

    # here the lemma of Gabel is flagged
    if task=='WORD_NAMING_WITH_DET' \
    and (Layers['lemma'].output[GABEL] > 0):
        flags['LEMMA'] = True
        
    return flags


In [ ]:
#/*********************
# * UPDATING ROUTINES *
# *********************/
def update_network(task, pos, Layers, W, flags):
    
    for x in Layers.keys():
        Layers[x].clear_input()

    Layers = get_external_input(task, pos, Layers, W, flags)
    Layers = get_internal_input(Layers, W)
    Layers = update_activation_of_nodes(Layers, W)
    return Layers


def set_input_to_zero(concept, lemma, gender, button):
    concept.clear_input()
    lemma.clear_input()
    gender.clear_input()
    button.clear_input()
    
    return concept, lemma, gender, button


def get_external_input(task, pos, Layers, W, flags):
    
    # picture input
    if pos==0 and (task=='PICTURE_NAMING' or task=='PICTURE_CLASSIFICATION' or task=='PICTURE_NAMING_fMRI'):
        Layers['concept'].input[FORK] += PREV_BIAS + EXTIN

    if pos==1 and (task=='PICTURE_NAMING' or task=='PICTURE_CLASSIFICATION' or task=='PICTURE_NAMING_fMRI'):
        Layers['concept'].input[FORK]  += PREV_BIAS + EXTIN
        Layers['concept'].input[SPOON] += PREV_BIAS + BIAS   # bias position "2"

    if pos==2 and (task=='PICTURE_NAMING' or task=='PICTURE_CLASSIFICATION' or task=='PICTURE_NAMING_fMRI'):
        Layers['concept'].input[FORK] += PREV_BIAS + EXTIN
        Layers['concept'].input[SPOON] += PREV_BIAS + BIAS   # bias position "2"
        Layers['concept'].input[CUP] += (PREV_BIAS + BIAS)   # bias position "3"

    if pos==3 and (task=='PICTURE_NAMING' or task=='PICTURE_CLASSIFICATION' or task=='PICTURE_NAMING_fMRI'):
        Layers['concept'].input[FORK] += PREV_BIAS + EXTIN
        Layers['concept'].input[SPOON] += PREV_BIAS + BIAS  # bias position "2"
        Layers['concept'].input[CUP] += PREV_BIAS + BIAS    # bias position "3"
        Layers['concept'].input[GLASS] += PREV_BIAS + BIAS  # bias position "4"

    if pos==4 and (task=='PICTURE_NAMING' or task=='PICTURE_CLASSIFICATION' or task=='PICTURE_NAMING_fMRI'): 
        Layers['concept'].input[FORK] += PREV_BIAS + EXTIN
        Layers['concept'].input[SPOON] += PREV_BIAS + BIAS    # bias position "2"
        Layers['concept'].input[CUP] += PREV_BIAS + BIAS      # bias position "3"
        Layers['concept'].input[GLASS] += PREV_BIAS + BIAS    # bias position "4"
        Layers['concept'].input[KNIFE] += PREV_BIAS + BIAS    # bias position "5"

    # Target concept gets activation enhancement
    if (task=='PICTURE_NAMING' or task=='PICTURE_NAMING_fMRI') and flags['ENHANCE_CONCEPT']:
        Layers['concept'].input[FORK] += 1.0 * EXTIN

    # word input
    if task=='WORD_NAMING' or task=='WORD_NAMING_WITH_DET':
        Layers['lemma'].input[GABEL] += EXTIN

    if pos==1 and task=='WORD_NAMING_WITH_DET':
        Layers['concept'].input[SPOON] += BIAS  # bias position "2"

    if pos==2 and task=='WORD_NAMING_WITH_DET':
        Layers['concept'].input[SPOON] += BIAS  # bias position "2"
        Layers['concept'].input[CUP] += BIAS    # bias position "3"

    if pos==3 and task=='WORD_NAMING_WITH_DET':
        Layers['concept'].input[SPOON] += BIAS  # bias position "2"
        Layers['concept'].input[CUP] += BIAS    # bias position "3"
        Layers['concept'].input[GLASS] += BIAS  # bias position "4"

    if pos==4 and task=='WORD_NAMING_WITH_DET':
        Layers['concept'].input[SPOON] += BIAS  # bias position "2"
        Layers['concept'].input[CUP] += BIAS    # bias position "3"
        Layers['concept'].input[GLASS] += BIAS  # bias position "4"
        Layers['concept'].input[KNIFE] += BIAS  # bias position "5"

    return Layers


def get_internal_input(Layers, W):

    for i in range(N_CONCEPTs):
        for j in range(N_CONCEPTs):
            Layers['concept'].input[i] +=  (Layers['concept'].output[j] * W['CONC'][j][i]) + (Layers['lemma'].output[j] * W['LEM'][j][i]) 

    #print('lemma    input', end=""); print(Layers['lemma'].input)
    #print('lemma   output', end=""); print(Layers['lemma'].output)
    #print('concept  input', end=''); print(Layers['concept'].input)
    #print('concept output', end=''); print(Layers['concept'].output)
    #print('W:\n', W['LEM'])
    for i in range(N_LEMMAs):
        for j in range(N_CONCEPTs):
            Layers['lemma'].input[i] += Layers['concept'].output[j] * W['LEM'][j][i]

    #print('lemma    input', end=""); print(Layers['lemma'].input)
    #print('lemma   output', end=""); print(Layers['lemma'].output)

    for i in range(N_GENDERs):
        for j in range(N_LEMMAs):
            Layers['gender'].input[i] += Layers['lemma'].output[j] * W['GEN'][j][i]

    for i in range(N_BUTTONs):
        for j in range(N_CONCEPTs):
            Layers['button'].input[i] += Layers['concept'].output[j] * W['BUTTON'][j][i]

    return Layers


def update_activation_of_nodes(Layers, W):
    Layers['concept'].update()
    Layers['lemma'].update()
    Layers['gender'].update()
    Layers['button'].update()
    
    return Layers


In [ ]:
#/*********************************************************
# *  HAZARD-, CUMUL_SURVIVAL-, MASS-FUNCTION ROUTINES     *
# *********************************************************/

def compute_hazard_rate(T, task, flags, Layers, W, hazard, CRIT_DIFF):

    if T >= 0:
        tau = int(T/STEP_SIZE) + 1

        if task=='PICTURE_NAMING' or task=='PICTURE_NAMING_fMRI':
            if flags['LEMMA'] \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[LOFFEL]) > CRIT_DIFF) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[TASSE]) > CRIT_DIFF) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[GLAS])  > CRIT_DIFF) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[MESSER]) > CRIT_DIFF):
                hazard[tau] = Layers['lemma'].output[GABEL] / Layers['lemma'].output.sum()
                # the denominator consists of semantic cohort memmbers
            else:
                hazard[tau] = 0.

        elif task == 'WORD_NAMING':
            if flags['LEMMA'] \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[LOFFEL]) > CRIT_DIFF) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[TASSE]) > CRIT_DIFF) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[GLAS])  > CRIT_DIFF) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[MESSER]) > CRIT_DIFF):
                hazard[tau] = Layers['lemma'].output[GABEL] / Layers['lemma'].output.sum()
                # the denominator consists of semantic cohort memmbers 
            else:
                hazard[tau] = 0.

        elif task == 'WORD_NAMING_WITH_DET':
            if (flags['LEMMA'] and flags['GENDER']) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[LOFFEL]) > CRIT_DIFF) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[TASSE]) > CRIT_DIFF) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[GLAS]) > CRIT_DIFF) \
            and ((Layers['lemma'].output[GABEL] - Layers['lemma'].output[MESSER]) > CRIT_DIFF) \
            and ((Layers['gender'].output[FEM]) > CRIT_DIFF):
                hazard[tau] = Layers['lemma'].output[GABEL] / Layers['lemma'].output.sum()
            else:
                hazard[tau] = 0.


        elif task == 'PICTURE_CLASSIFICATION':
            if flags['BUTTON_NODE'] \
            and (Layers['button'].output[B_MANMADE] - Layers['button'].output[B_NATURAL] > CRIT_DIFF):
                # In simulating picture classification with vocal
                # responding (Riley et al. 2015, FiP), the buttons would
                # become the corresponding lemmas
                hazard[tau] = 1.
            else:
                hazard[tau] = 0.
            
    return hazard


def compute_cumul_survival_function(hazard, Survival):
    """NOTE: cum_survival or S[s] is upto and including s"""
    for s in range(N_STEPs):
        aux = 1.0
        for j in range(s+1):
            aux *= (1.0 - hazard[j])
        Survival[s] = aux
    return Survival


def compute_mass_function(mass, hazard, Survival):
    """
    NOTE: Prob(not selected before step s) equals S[s-1], that is
    surviving upto and including the previous time step
    """
    # NOTE: f[0] will always be 0, so does not have to be computed
    for s in range(1, N_STEPs):
        mass[s] = hazard[s] * Survival[s-1]
    return mass


def compute_expectation_of_RT_(mass):
    mean = 0.
    for s in range(N_STEPs):
        mean += mass[s] * s * STEP_SIZE

    return mean


def reset_f_h_S_(mass_f, hazard, Survival):
    mass_f.fill(0.); hazard.fill(0.); Survival.fill(0.)
    #for s in range(N_STEPs):
    #    f[s] = 0.0
    #    h[s] = 0.0
    #    S[s] = 0.0
    return mass_f, hazard, Survival

        
def compute_BOLD(task, SIM, BOLD):
    i_task = tasks.index(task)
    for j in range(N_POSs):
        # 6.0 means the intercept estimated from Canini et al. (2016)
        # 4.5 means slope estimated from Canini et al. (2016)
        BOLD[i_task][j] = 6.0 + (SIM[i_task][j] - SIM[i_task][0]) * 4.5 / (SIM[i_task][4] - SIM[i_task][0])
    
    return BOLD

In [ ]:
#/*****************
# *  I/0 ROUTINES *
# *****************/
def print_heading():
    print("Simulation of Lemma Retrieval in Speaking  (c) Ardi Roelofs")
    #print("working...\n")


def print_expectations_of_RT(task, SIM, BOLD, BLOCKED_CYCLIC=False, graph=False):

    if task == 'PICTURE_NAMING': 
        print("# Picture Naming", task)
    elif task == 'PICTURE_CLASSIFICATION': 
        print("\n# Picture - Semantic Classification", task)
    elif task == 'WORD_NAMING': 
        print("\n# Word Naming", task)
    elif task == 'WORD_NAMING_WITH_DET':
        print("\n# Word Naming With Gender-Marked Determiner (Transfer)", task)
    elif task == 'PICTURE_NAMING_fMRI': 
        print("\n# Picture Naming fMRI", task)

    i_task = tasks.index(task)
    print("  POSITION   LATENCY    EFFECT")
    for j in range(N_POSs):
        print('{:6d} '.format(j+1), end='')
        print("       {0:3.0f}        {1:3.0f}".format(SIM[i_task][j], SIM[i_task][j] - SIM[i_task][0]))
    
    tot_latency = 0.
    tot_effect = 0.
    if BLOCKED_CYCLIC:
        for j in range(N_POSs):
            tot_latency += SIM[i_task][j]
            tot_effect += SIM[i_task][j] - SIM[i_task][0]
        #print("Cycle mean  {0:6d}    {1:6d}".format(tot_latency/5, tot_effect/5)) 
        print("Cycle mean  {0}    {1}".format(tot_latency/5, tot_effect/5)) 

    if task == 'PICTURE_NAMING_fMRI':
        print("  POSITION   BOLD")
        for j in range(N_POSs):
            print("{:6d}".format(j+1), end="")
            print("     {:6.2f}".format(BOLD[i_task][j]))

    if task == 'PICTURE_NAMING':
        print("\n  Parameter values:")
        print("  concept_rate  : {:.4f} [prop/ms]".format(CONC_rate/STEP_SIZE))
        print("  lemma_rate    : {:.4f} [prop/ms]".format(LEM_rate/STEP_SIZE))
        print("  external input: {:.4f} [act_units/ms]".format(EXTIN/STEP_SIZE))
        print("  decay rate    : {:.4f} [prop/ms]".format(DECAY_rate/STEP_SIZE))
        print("  crit_diff     : {:.4f} [act_units]".format(CRIT_DIFF))
        print("  bias          : {:.4f} [act_units]".format(BIAS))
        
        
    if graph:
        X = np.zeros((N_POSs), dtype=np.int)
        _max, _min = np.iinfo(np.uint64).min, np.iinfo(np.uint64).max
        for j in range(N_POSs):
            _X = SIM[i_task][j] - SIM[i_task][0]
            if _X > _max:
                _max = _X
            if _X < _min:
                _min = _X
            X[j] = _X
        #print('X:', X)
        plt.plot(X, '-', X, 'go')
        plt.axis([-0.5, 4.5, _min-50, _max+50])
        plt.ylabel('Latency effect (ms)')
        plt.xlabel('Ordinal postion within category')
        plt.title('{0:}'.format(task))
        #plt.legend()
        plt.show()
        
    #printf("\npress any key to continue ");
    #getchar();


In [ ]:
#CRITICAL_DIFFS['CD_PN'] = 0.1
#CRITICAL_DIFFS['CD_PC'] = 0.2
#CRITICAL_DIFFS['CD_WN'] = 0.01
#CRITICAL_DIFFS
#main(Critical_Diffs=CRITICAL_DIFFS, graph=True)
main(graph=True)
#main()

# output expected
```text
Simulation of Lemma Retrieval in Speaking  (c) Ardi Roelofs

  Picture Naming

  POSITION   LATENCY    EFFECT
     1         106         0
     2         119        13
     3         130        24
     4         141        35
     5         150        44

Parameter values:
conc_rate : 0.0050 [prop/ms]
lem_rate  : 0.0074 [prop/ms]
exin      : 0.1965 [act_units/ms]
d         : 0.0240 [prop/ms]
cd        : 1.6000 [act_units]
bias      : 2.5000 [act_units]

  Picture - Semantic Classification
  POSITION   LATENCY    EFFECT
     1         250         0
     2         200       -50
     3         175       -75
     4         175       -75
     5         150      -100

  Word Naming
  POSITION   LATENCY    EFFECT
     1          50         0
     2          50         0
     3          50         0
     4          50         0
     5          50         0

  Word Naming With Gender-Marked Determiner (Transfer)
  POSITION   LATENCY    EFFECT
     1         100         0
     2         103         3
     3         107         7
     4         111        11
     5         115        15

  Picture Naming fMRI
  POSITION   LATENCY    EFFECT
     1         106         0
     2         119        13
     3         130        24
     4         141        35
     5         150        44
  POSITION   BOLD
     1      6.00   
     2      7.33   
     3      8.45   
     4      9.58   
     5      10.50   
```

# 以下は オリジナルソースコード

```C++
/***************************************************
 *                                                 *
 *  CONCEPTUAL BIAS.C                              *
 *                                                 *
 *  SIMULATION OF LEMMA RETRIEVAL IN SPEAKING      *
 *                                                 *
 *  Simulation of cumulative semantic and          *
 *  semantic blocking effects                      *
 *                                                 *
 *  Journal: Cognition                             *
 *                                                 *
 *  Ardi Roelofs, November 2017                    *
 *                                                 *
 *  (C) Copyright DCC                              *
 *                                                 *
 ***************************************************/

 /*
  A brief note on C and programming style:
 
  This program is written in the C programming language, which is
  described by Kernighan and Ritchie (1988) and many others. C was 
  chosen because it is among the most widely and frequently used 
  programming languages, and C programs can be compiled (using 
  freeware C and C++ compilers) for all main computer platforms 
  and operating systems. 
  
  Following Kernighan and Plauger's (1978) maxim "write clearly 
  -- don't be too clever", I avoided the use of pointers
  and other initially somewhat obscure constructs to help readers 
  unfamiliar with C. Given that the program is rather small 
  (everything is in one file), I chose for external variables 
  that are globally accessible to all functions rather than 
  using function arguments and return values for communicating
  data between functions. External variables are more convenient
  and efficient here than long argument lists.

  Kernighan, B.W., & Plauger, P.J. (1978). The Elements of
  Programming Style (Second Edition). New York: McGraw Hill.

  Kernighan, B.W., & Ritchie, D.M. (1988). The C Programming Language
  (Second Edition). Englewood Cliffs, NJ: Prentice Hall.
 */


#include <stdio.h>
#include <stdlib.h>
#include <float.h>


#define STEP_SIZE 25   /* duration time step in ms */
#define N_STEPs (1000+STEP_SIZE)/STEP_SIZE /* trunc after 41 steps */
#define N_CONCEPTs 6
#define N_LEMMAs 6
#define N_GENDERs 3
#define N_BUTTONs 2
#define N_TASKs 5
#define N_POSs 5 /* ordinal position*/
#define Y 1.0   /* connection present */
#define N 0.0   /* connection absent */
#define PICTURE_NAMING 0 /* Belke (2013, JML), Exp 1. */
#define PICTURE_CLASSIFICATION 1 /* manmade, natural */
#define WORD_NAMING 2
#define WORD_NAMING_WITH_DET 3
#define PICTURE_NAMING_fMRI 4
         /* BOLD response, Canini et al. (2016, HBM) */

 /* Labeling network nodes */
 /* Concepts */
#define FORK 0
#define SPOON 1
#define CUP 2
#define GLASS 3
#define KNIFE 4
#define MANMADE 5

/* German lemmas */
#define GABEL 0  /* fork */
#define LOFFEL 1 /* spoon */
#define TASSE 2  /* cup */
#define GLAS 3   /* glass */
#define MESSER 4 /* knife */
#define KUENSTLICH 5  /* manmade */

/* Grammatical genders */
#define MASC 0   /* masculine */
#define FEM 1    /* feminine */
#define NEUTER 2 /* neuter */

/* Button nodes */
#define B_MANMADE 0  /* Button 1: manmade */
#define B_NATURAL 1  /* Button 2: natural */

 double CONC_con[N_CONCEPTs][N_CONCEPTs];

 /* Connections conceptual network for categorical relations */
 double CATEG_con[N_CONCEPTs][N_CONCEPTs] =  {
	           /* FORK  SPOON CUP GLASS KNIFE MANMADE */
 /* FORK   */   {   N,    Y,   Y,    Y,    Y,    Y },
 /* SPOON  */   {   Y,    N,   Y,    Y,    Y,    Y },
 /* CUP    */   {   Y,    Y,   N,    Y,    Y,    Y },
 /* GLASS  */   {   Y,    Y,   Y,    N,    Y,    Y },
 /* KNIFE  */   {   Y,    Y,   Y,    Y,    N,    Y },
 /* MANMADE*/   {   Y,    Y,   Y,    Y,    Y,    N }
 };

 /* Connections conceptual network for thematic relations */
 double ASSOC_con[N_CONCEPTs][N_CONCEPTs] =  {
	          /* CHURCH RING DRESS CAKE BOUQUET WEDDING */
 /* CHURCH  */   {   N,    N,   N,    N,    N,    Y },
 /* RING    */   {   N,    N,   N,    N,    N,    Y },
 /* DRESS   */   {   N,    N,   N,    N,    N,    Y },
 /* CAKE    */   {   N,    N,   N,    N,    N,    Y },
 /* BOUQUET */   {   N,    N,   N,    N,    N,    Y },
 /* WEDDING */   {   Y,    Y,   Y,    Y,    Y,    N }
 };


 /* Connections between concept and lemma nodes */
 /* Lemma nodes are Gabel, Loffel, Tasse, Glas, Messer
    (and kuenstlich [manmade]) for the category "tableware" 
	of Belke (2013, JML), and Kirche, Ring, Brautkleid, 
	Torte, Blumenstrauss (and Hochzeit) for the theme 
	"wedding" of Rose & Abdel Rahman (2016, Cognition)
*/
 double LEM_con[N_CONCEPTs][N_LEMMAs] = {
    { Y,  N,  N,  N,  N,  N },
    { N,  Y,  N,  N,  N,  N },
    { N,  N,  Y,  N,  N,  N },
    { N,  N,  N,  Y,  N,  N },
    { N,  N,  N,  N,  Y,  N },
    { N,  N,  N,  N,  N,  Y }
 };

 /* Connections between lemma and gender nodes */
 double GEN_con[N_LEMMAs][N_GENDERs] =  {
	               /* MASC  FEM  NEUTER */
 /* GABEL  */      {   N,    Y,    N  },
 /* LOFFEL */      {   Y,    N,    N  },
 /* TASSE  */      {   N,    Y,    N  },
 /* GLAS   */      {   N,    N,    Y  },
 /* MESSER */      {   N,    N,    Y  },
 /* KUENSTLICH*/   {   N,    N,    N  }
 };


 /* Connections between concept and button nodes */
 double BUTTON_con[N_CONCEPTs][N_BUTTONs] = {
	        /* B_MANMADE B_NATURAL */
	 /* FORK   */ { N,    N },
	 /* SPOON  */ { N,    N },
	 /* CUP    */ { N,    N },
	 /* GLASS  */ { N,    N },
	 /* KNIFE  */ { N,    N },
	 /* MANMADE*/ { Y,    N }
 };




 /* Here a network node is defined, having an activation level,
 and an input buffer; its output is equal to its activation */
 struct node { 
	 double act;
	 double input;
 } concept[N_CONCEPTs], lemma[N_LEMMAs], gender[N_GENDERs], button[N_BUTTONs];

 
 int pos;   /* ordinal position, i.e., 1 to 5 (in C is 0 to 4) */
 int T;     /* time in ms */

 int task;  /* PICTURE NAMING, PICTURE CLASSIFICATION, WORD NAMING, 
			   WORD NAMIMG WITH DET, PICTURE_NAMING_fMRI */

 /* Parameter set plus values */
 double CONC_rate = 0.5 * 0.0101 * STEP_SIZE;   
            /* 0.5 of default prop per step_size ms 
			   because of the increased size of conceptual
			   network, cf. Roelofs (2003, Psych. Rev.) */
 double LEM_rate = 0.0074 * STEP_SIZE;   /* prop per step_size ms */
 double DECAY_rate = 0.0240 * STEP_SIZE; /* prop per step_size ms */
 double EXTIN = 0.1965 * STEP_SIZE;      /* act_units per step_size ms */
 double CRIT_DIFF;                       /* act_units */

 double BIAS = 2.5;

 int BLOCKED_CYCLIC = 0; /* 1 = run for blocked-cyclic naming */
 double PREV_BIAS = 0.0; /* previous bias */ 
 /* The PREV_BIAS parameter is needed for simulating blocked-cyclic
    naming, because the bias increase during a cycle is also present 
	during the next cycle. To run the model for blocked-cyclic naming,
	set the PREV_BIAS values by hand for each new cycle.
    Previous bias in cycle 1: 0.0, cycle 2: 2.5, cycle 3: 5.0, 
	                 cycle 4: 7.5, cycle 5: 10.0, cycle 6: 12.5
	The semantic blocking effect in a cycle is the mean increase over
	ordinal positions relative to the first position. The effect of bias
	for the first ordinal position is the same for the homogeneous and 
	the heterogeneous conditions. For later ordinal positions, the bias 
	effect will differ between conditions because the position effect
	occurs only within categories or themes (thus only in the homogeneous
	condition).
  */

 int TAXONOMIC = 1; /* categorical relatedness, Belke (2013, JML) */
 int THEMATIC = 0;  /* thematic relatedness, 
					   Rose & Abdel Rahman (2016, Cognition) */
 
 /* The parameter values for the critical difference [selection
    threshold] for the different tasks: PN = picture naming, 
	PC is picture classification, WR is word reading */
  double CD_PN = 1.6, CD_PC = 1.6,  CD_WR = 1.6; 
  /* default: all values the same */

 /* control flags for production rule application */
 /* cf. EPIC, Meyer & Kieras, Psych. Rev. 1997 */  
 int FLAG_CONCEPT;
 int ENHANCE_CONCEPT;
 int FLAG_LEMMA;
 int FLAG_GENDER;
 int FLAG_CATEGORY_CONCEPT;
 int FLAG_BUTTON_NODE;

 int SIM_DATA[N_TASKs][N_POSs];
 double BOLD_response[N_TASKs][N_POSs];

 double h[N_STEPs];
 double S[N_STEPs];
 double f[N_STEPs];

 void reset_network(void);
 void set_spreading_rates(void);
 void reset_spreading_rates(void);
 void set_task_dependent_parameters(void);
 void update_network(void);
 void set_input_to_zero(void);
 void get_external_input(void);
 void get_internal_input(void);
 void update_activation_of_nodes(void);

 void reset_system(void);
 void reset_f_h_S_(void);
 void compute_POS_functions(void);
 void compute_hazard_rate(void);
 void compute_cumul_survival_function(void);
 void compute_mass_function(void);
 double compute_expectation_of_RT_(void);
 void compute_BOLD_response(void);

 void reset_production_rule_system(void);
 void apply_production_rules(void);

 void print_heading(void);
 void print_expectations_of_RT(void);
  

/*****************
 * MAIN ROUTINES *
 *****************/

 main()
 {
    for(task=0;task<N_TASKs;task++) {

       reset_system();
       set_spreading_rates();
       set_task_dependent_parameters();

	   print_heading();

       compute_POS_functions();

	   print_expectations_of_RT();

       reset_spreading_rates();
    }
 }


 void compute_POS_functions()
 {

	 for (pos = 0; pos < N_POSs; pos++) {
		 reset_f_h_S_();
		 reset_network();
		 reset_production_rule_system();
		 for (T = 0; T < 1000; T += STEP_SIZE) {
			 /* stop for lemma retrieval latency > 1000 ms */
			 compute_hazard_rate();
			 update_network();
			 apply_production_rules();
		 }
		 compute_cumul_survival_function();
		 compute_mass_function();
		 SIM_DATA[task][pos] = (int)compute_expectation_of_RT_();
	 }
	      if(task==PICTURE_NAMING_fMRI)
		   compute_BOLD_response();
 }



 void reset_system()
 {
     int j;

      for(j=0;j<N_POSs;j++)
    	SIM_DATA[task][j]=0;

      for(j=0;j<N_POSs;j++)
    	BOLD_response[task][j]=0.0;

 }


 void set_spreading_rates()
 {
   int i,j;

  /* Sets the conceptual connections for categorical reations */
   if(TAXONOMIC) {
   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_CONCEPTs;j++) 
	   CONC_con[i][j] = CATEG_con[i][j];
   }

 /* Sets the conceptual connections for thematic reations */
   if(THEMATIC) {
   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_CONCEPTs;j++) 
	   CONC_con[i][j] = ASSOC_con[i][j];
   }


 /* Sets the strengths of the connections */

   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_CONCEPTs;j++)
	   CONC_con[i][j]*=CONC_rate;

   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_LEMMAs;j++) 
	   LEM_con[i][j]*=LEM_rate;

   for(i=0;i<N_LEMMAs;i++)
     for(j=0;j<N_GENDERs;j++) 
	   GEN_con[i][j]*=LEM_rate;

   for(i=0;i<N_CONCEPTs;i++)
	 for(j=0;j<N_BUTTONs;j++)
		BUTTON_con[i][j]*=LEM_rate;

 }


 void set_task_dependent_parameters()
 {

    if(task==PICTURE_NAMING) {
       CRIT_DIFF=CD_PN;
       }

    if(task==PICTURE_CLASSIFICATION) {
       CRIT_DIFF=CD_PC;
       }

    if(task==WORD_NAMING) {
       CRIT_DIFF=CD_WR;
       }

    if(task==WORD_NAMING_WITH_DET) {
       CRIT_DIFF=CD_WR;
       }

    if(task==PICTURE_NAMING_fMRI) {
       CRIT_DIFF=CD_PN;
       }
 }


 void reset_spreading_rates()
 {
 /* Needed, because otherwise in later tasks rates become a
    proportion of earlier ones; this routine keeps them fixed */

   int i,j;

   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_CONCEPTs;j++) 
	   CONC_con[i][j]*=(1.0/CONC_rate);

   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_LEMMAs;j++)
	   LEM_con[i][j]*=(1.0/LEM_rate);


   for(i=0;i<N_LEMMAs;i++)
     for(j=0;j<N_GENDERs;j++) 
  	   GEN_con[i][j]*=(1.0/LEM_rate);
	
   for(i=0;i<N_CONCEPTs;i++)
	 for(j=0;j<N_BUTTONs;j++)
		BUTTON_con[i][j]*=(1.0/LEM_rate);

 }

 void reset_network()
 {
   int i;

   for(i=0;i<N_CONCEPTs;i++) 
     concept[i].act=0.0;

   for(i=0;i<N_LEMMAs;i++)
     lemma[i].act=0.0;

   for(i=0;i<N_GENDERs;i++)
	 gender[i].act=0.0;

   for(i=0;i<N_BUTTONs;i++)
	 button[i].act=0.0;

}


 void reset_production_rule_system()
 {
  /* reset control flags in working memory */
  
  FLAG_CONCEPT = 0;
  ENHANCE_CONCEPT = 0;
  FLAG_LEMMA = 0;

  FLAG_GENDER = 0;

  FLAG_CATEGORY_CONCEPT = 0;
  FLAG_BUTTON_NODE = 0;
  
 }




/*********************
 * UPDATING ROUTINES *
 *********************/

 void update_network()
 {
   set_input_to_zero();
   get_external_input();
   get_internal_input();
   update_activation_of_nodes();
 }


 void apply_production_rules()
 {

 /* Production rule application for picture naming */

 /* here the lemma of Gabel for the selected concept FORK is flagged */
  if((task==PICTURE_NAMING || task==PICTURE_NAMING_fMRI) 
	  && FLAG_CONCEPT==1 && lemma[GABEL].act) 
     FLAG_LEMMA = 1;

 /* here the concept FORK is flagged for selection and enhancement 
              cf. P1 in Roelofs, 2003, Psych. Rev., p. 100 */
  if((task==PICTURE_NAMING || task==PICTURE_NAMING_fMRI) 
	  && concept[FORK].act) {
     FLAG_CONCEPT = 1;
     ENHANCE_CONCEPT = 1;
  }

 /* production rule application for picture classification */

 /* here the button node for the category concept MAN-MADE is flagged */
  if(task==PICTURE_CLASSIFICATION  
	  && FLAG_CATEGORY_CONCEPT==1 && button[B_MANMADE].act) 
     FLAG_BUTTON_NODE = 1;

 /* here the category concept MAN-MADE for the concept FORK is flagged */
  if(task==PICTURE_CLASSIFICATION  
	  && FLAG_CONCEPT==1 && concept[MANMADE].act) 
     FLAG_CATEGORY_CONCEPT = 1;

 /* here the concept FORK is flagged */
  if(task==PICTURE_CLASSIFICATION  && concept[FORK].act) 
     FLAG_CONCEPT = 1;


 /* Production rule application for word reading */

 /* here the lemma of Gabel is flagged */
  if(task==WORD_NAMING && lemma[GABEL].act) 
     FLAG_LEMMA = 1;


 /* Production rule application for word reading 
                     with gender-marked determiner */

 /* here the gender of the lemma of Gabel is flagged */
  if(task==WORD_NAMING_WITH_DET && FLAG_LEMMA == 1 && gender[FEM].act) 
     FLAG_GENDER = 1;

 /* here the lemma of Gabel is flagged */
  if(task==WORD_NAMING_WITH_DET && lemma[GABEL].act) 
     FLAG_LEMMA = 1;
  
 }



 void set_input_to_zero()
 {
   int i;

   for(i=0;i<N_CONCEPTs;i++)
       concept[i].input=0.0;

   for(i=0;i<N_LEMMAs;i++)
       lemma[i].input=0.0;
 
   for(i=0;i<N_GENDERs;i++)
	   gender[i].input=0.0;

   for(i=0;i<N_BUTTONs;i++)
	   button[i].input=0.0;

 }



  void get_external_input()
 {


      /* picture input */
	 
		if(pos==0 && (task==PICTURE_NAMING || task==PICTURE_CLASSIFICATION
			|| task==PICTURE_NAMING_fMRI)) {
 	   concept[FORK].input += (PREV_BIAS + EXTIN);  
		} 

		if(pos==1 && (task==PICTURE_NAMING || task==PICTURE_CLASSIFICATION
			|| task==PICTURE_NAMING_fMRI)) {
	   concept[FORK].input += (PREV_BIAS + EXTIN);
	   concept[SPOON].input += (PREV_BIAS + BIAS); /* bias position "2" */
  
		} 

		if(pos==2 && (task==PICTURE_NAMING || task==PICTURE_CLASSIFICATION
			|| task==PICTURE_NAMING_fMRI)) {
	   concept[FORK].input += (PREV_BIAS + EXTIN); 
	   concept[SPOON].input += (PREV_BIAS + BIAS); /* bias position "2" */ 
	   concept[CUP].input += (PREV_BIAS + BIAS);   /* bias position "3" */
		} 

		if(pos==3 && (task==PICTURE_NAMING || task==PICTURE_CLASSIFICATION
			|| task==PICTURE_NAMING_fMRI)) {
	   concept[FORK].input += (PREV_BIAS + EXTIN);
	   concept[SPOON].input += (PREV_BIAS + BIAS); /* bias position "2" */ 
	   concept[CUP].input += (PREV_BIAS + BIAS);   /* bias position "3" */ 
	   concept[GLASS].input += (PREV_BIAS + BIAS); /* bias position "4" */
		} 

		if(pos==4 && (task==PICTURE_NAMING || task==PICTURE_CLASSIFICATION
			|| task==PICTURE_NAMING_fMRI)) {
	   concept[FORK].input += (PREV_BIAS + EXTIN);  
	   concept[SPOON].input += (PREV_BIAS + BIAS);  /* bias position "2" */ 
	   concept[CUP].input += (PREV_BIAS + BIAS);    /* bias position "3" */ 
	   concept[GLASS].input += (PREV_BIAS + BIAS);  /* bias position "4" */ 
	   concept[KNIFE].input += (PREV_BIAS + BIAS);  /* bias position "5" */
		} 

	/* Target concept gets activation enhancement */
      if((task==PICTURE_NAMING || task==PICTURE_NAMING_fMRI) 
		    && ENHANCE_CONCEPT==1 )
	   concept[FORK].input += 1.0 * EXTIN;



      /* word input */

 	  if(task==WORD_NAMING || task==WORD_NAMING_WITH_DET) {
 	   lemma[GABEL].input += EXTIN;  
		} 

		if(pos==1 && task==WORD_NAMING_WITH_DET) {
	   concept[SPOON].input += BIAS; /* bias position "2" */ 
		} 

		if(pos==2 && task==WORD_NAMING_WITH_DET) {
	   concept[SPOON].input += BIAS;   /* bias position "2" */ 
	   concept[CUP].input += BIAS;     /* bias position "3" */ 
		} 

		if(pos==3 && task==WORD_NAMING_WITH_DET) {
	   concept[SPOON].input += BIAS;  /* bias position "2" */ 
	   concept[CUP].input += BIAS;    /* bias position "3" */ 
	   concept[GLASS].input += BIAS;  /* bias position "4" */ 
 		} 

		if(pos==4 && task==WORD_NAMING_WITH_DET) {
	   concept[SPOON].input += BIAS;  /* bias position "2" */ 
	   concept[CUP].input += BIAS;    /* bias position "3" */ 
	   concept[GLASS].input += BIAS;  /* bias position "4" */ 
	   concept[KNIFE].input += BIAS;  /* bias position "5" */ 
		} 

 }


 void get_internal_input()
 {
   int i,j;

   for(i=0;i<N_CONCEPTs;i++)
     for(j=0;j<N_CONCEPTs;j++) 
       concept[i].input += ( (concept[j].act * CONC_con[j][i])
		      +
		    (lemma[j].act * LEM_con[j][i])  );


   for(i=0;i<N_LEMMAs;i++)
     for(j=0;j<N_CONCEPTs;j++) 
       lemma[i].input += ( concept[j].act * LEM_con[j][i] );
       

   for(i=0;i<N_GENDERs;i++)
     for(j=0;j<N_LEMMAs;j++) 
       gender[i].input += ( lemma[j].act * GEN_con[j][i] );

   for(i=0;i<N_BUTTONs;i++)
	 for(j=0;j<N_CONCEPTs;j++)
		button[i].input += (concept[j].act * BUTTON_con[j][i]);

 }


 void update_activation_of_nodes()
 {
   int i;

   for(i=0;i<N_CONCEPTs;i++) 
     concept[i].act = ((concept[i].act * (1.0 - DECAY_rate)) 
		 + concept[i].input);

   for(i=0;i<N_LEMMAs;i++) 
     lemma[i].act = ((lemma[i].act * (1.0 - DECAY_rate)) 
		 + lemma[i].input);
     
   for(i=0;i<N_GENDERs;i++) 
     gender[i].act = ((gender[i].act * (1.0 - DECAY_rate)) 
		 + gender[i].input);

   for(i=0;i<N_BUTTONs;i++)
	   button[i].act = ((button[i].act * (1.0 - DECAY_rate))
		   + button[i].input);

 }



/*********************************************************
 *  HAZARD-, CUMUL_SURVIVAL-, MASS-FUNCTION ROUTINES     *
 *********************************************************/


 void compute_hazard_rate()
 {


   if(T>=0) {

      if(task==PICTURE_NAMING || task==PICTURE_NAMING_fMRI) {

	 if( FLAG_LEMMA==1   
	    &&
	   ( (lemma[GABEL].act - lemma[LOFFEL].act) > CRIT_DIFF )
	    &&
	   ( (lemma[GABEL].act - lemma[TASSE].act) > CRIT_DIFF )
	    && 
	   ( (lemma[GABEL].act - lemma[GLAS].act)  > CRIT_DIFF )
	    && 
	   ( (lemma[GABEL].act - lemma[MESSER].act) > CRIT_DIFF )  )

	    h[(T/STEP_SIZE)+1] = lemma[GABEL].act / 
			        (lemma[GABEL].act   
			       + lemma[LOFFEL].act  
			       + lemma[TASSE].act
			       + lemma[GLAS].act
			       + lemma[MESSER].act );
		   /* the denominator consists of semantic cohort memmbers */
	 else
	    h[(T/STEP_SIZE)+1] = 0.0;
      }

      if(task==WORD_NAMING) {

		  if (FLAG_LEMMA == 1
			  &&
			  ((lemma[GABEL].act - lemma[LOFFEL].act) > CRIT_DIFF)
			  &&
			  ((lemma[GABEL].act - lemma[TASSE].act) > CRIT_DIFF)
			  &&
			  ((lemma[GABEL].act - lemma[GLAS].act)  > CRIT_DIFF)
			  &&
			  ((lemma[GABEL].act - lemma[MESSER].act) > CRIT_DIFF))

			  h[(T / STEP_SIZE) + 1] = lemma[GABEL].act /
			       (lemma[GABEL].act
				  + lemma[LOFFEL].act
				  + lemma[TASSE].act
				  + lemma[GLAS].act
				  + lemma[MESSER].act);
		  /* the denominator consists of semantic cohort memmbers */
		  else
			  h[(T / STEP_SIZE) + 1] = 0.0;
	  }


    if(task==WORD_NAMING_WITH_DET) {

	 if( FLAG_LEMMA==1 && FLAG_GENDER==1   
	    &&
		 ((lemma[GABEL].act - lemma[LOFFEL].act) > CRIT_DIFF)
		 &&
		 ((lemma[GABEL].act - lemma[TASSE].act) > CRIT_DIFF)
		 &&
		 ((lemma[GABEL].act - lemma[GLAS].act)  > CRIT_DIFF)
		 &&
		 ((lemma[GABEL].act - lemma[MESSER].act) > CRIT_DIFF)
		 &&
	    (gender[FEM].act ) > CRIT_DIFF )

		 h[(T / STEP_SIZE) + 1] = lemma[GABEL].act /
		      (lemma[GABEL].act
			 + lemma[LOFFEL].act
			 + lemma[TASSE].act
			 + lemma[GLAS].act
			 + lemma[MESSER].act);
	 else
	    h[(T/STEP_SIZE)+1] = 0.0;
      }



      if(task==PICTURE_CLASSIFICATION) {

	 if( FLAG_BUTTON_NODE==1   
	    &&
	   ( button[B_MANMADE].act - button[B_NATURAL].act > CRIT_DIFF ) )
		 /* In simulating picture classification with vocal
		 responding (Riley et al. 2015, FiP), the buttons would
		 become the corresponding lemmas
		 */
		 h[(T/STEP_SIZE)+1] = 1.0;
	 else
	    h[(T/STEP_SIZE)+1] = 0.0;
      }


   }
 }


 void compute_cumul_survival_function()
 {
     int j,s;
     double aux;

     /* NOTE: cum_survival or S[s] is upto and including s */

     for(s=0;s<N_STEPs;s++) {
       for(j=0, aux=1.0;j<=s;j++)
	  aux*=(1.0-h[j]);
       S[s]=aux;
       }
 }

 void compute_mass_function()
 {
     int s;

     /* NOTE: Prob(not selected before step s) equals S[s-1], that is
	surviving upto and including the previous time step */

     for(s=1;s<N_STEPs;s++)
       f[s]=h[s] * S[s-1];

     /* NOTE: f[0] will always be 0, so does not have to be computed */
 }

 double compute_expectation_of_RT_()
 {
     int s;
     double mean=0.0;

	 for(s=0;s<N_STEPs;s++) 
	  mean+=f[s] * s * STEP_SIZE;
	 
     return mean;
 }

 void reset_f_h_S_()
 {
    int s;

    for(s=0;s<N_STEPs;s++) {
      f[s]=0.0;
      h[s]=0.0;
      S[s]=0.0;
      }
 }

 void compute_BOLD_response()
 {
    int j;

    for(j=0;j<N_POSs;j++)
     BOLD_response[task][j]= 
	  (6.0 /* 6.0 = intercept estimated from Canini et al. (2016) */
	 + ( (double) SIM_DATA[task][j] - (double) SIM_DATA[task][0] ) 
	 * (4.5 / ( (double) SIM_DATA[task][4] - (double) SIM_DATA[task][0])));
	       /* 4.5 = slope estimated from Canini et al. (2016) */
  }



/*****************
 *  I/0 ROUTINES *
 *****************/

 void print_heading()
 {

   printf("Simulation of Lemma Retrieval in Speaking  (c) Ardi Roelofs\n");
   printf("working...\n");

 }


 void print_expectations_of_RT()
 {
     int j, tot_latency=0, tot_effect=0;

     if(task==PICTURE_NAMING)
	printf("\n  Picture Naming\n\n");
     if(task==PICTURE_CLASSIFICATION)
	printf("\n  Picture - Semantic Classification\n\n");
     if(task==WORD_NAMING)
	printf("\n  Word Naming\n\n");
     if(task==WORD_NAMING_WITH_DET)
	printf("\n  Word Naming With Gender-Marked Determiner (Transfer)\n\n");
     if(task==PICTURE_NAMING_fMRI)
	printf("\n  Picture Naming fMRI\n\n");

     printf("  POSITION   LATENCY    EFFECT\n");
     for(j=0;j<N_POSs;j++) {
	 printf("%6d ", j+1);
	 printf("     %6d    %6d", SIM_DATA[task][j], 
		 SIM_DATA[task][j] - SIM_DATA[task][0] );
	 printf("\n");
     }

	 if(BLOCKED_CYCLIC) {
       for(j=0;j<N_POSs;j++) {
         tot_latency+=SIM_DATA[task][j];
         tot_effect+=SIM_DATA[task][j] - SIM_DATA[task][0];
 	      }
	 printf("Cycle mean  %6d    %6d\n", tot_latency/5, tot_effect/5); 
	 }

	 if(task==PICTURE_NAMING_fMRI) {  
     printf("  POSITION   BOLD\n");
	 for(j=0;j<N_POSs;j++) {
	 printf("%6d ", j+1);        
	 printf("     %.2f   ",   BOLD_response[task][j]);
	 printf("\n");              
      }
	 }

     printf("\nParameter values:\n");
     printf("conc_rate : %.4f [prop/ms]\n",CONC_rate/STEP_SIZE);
     printf("lem_rate  : %.4f [prop/ms]\n",LEM_rate/STEP_SIZE);
     printf("exin      : %.4f [act_units/ms]\n",EXTIN/STEP_SIZE);
     printf("d         : %.4f [prop/ms]\n",DECAY_rate/STEP_SIZE);
     printf("cd        : %.4f [act_units]\n",CRIT_DIFF);
     printf("bias      : %.4f [act_units]\n",BIAS);

     printf("\npress any key to continue ");

     getchar();
 }
```

In [ ]:
import svgutils.transform as sg
from IPython.display import SVG,display

#create new SVG figure
fig = sg.SVGFigure("16cm", "10cm")

# load matpotlib-generated figures
fig1 = sg.fromfile('../figures/2018Roelofs_fig1.svg')
plot1 = fig1.getroot()
fig.append([plot1])
fig.save("svg_filter_line2.svg")
display(SVG(filename='svg_filter_line2.svg'))

In [ ]:
!gls -tl '../figures/2018Roelofs_fig1.svg'